In [1]:
from collections import defaultdict
import pandas as pd
import math
df = pd.read_csv("csi_failure_dataset_open_source_systems.csv")
df = df.fillna('')

## Section 4: Table 1

In [2]:
upstream_systems = list(df['Upstream System'])
downstream_systems = list(df['Downstream System'])
dic = defaultdict(int)
for systems in list(zip(upstream_systems, downstream_systems)):
    dic[systems[0] + "_" + systems[1]] += 1
system_order = [["Spark", "Hive"], ["Spark", "YARN"], ["Spark", "HDFS"], ["Spark", "Kafka"],
                ["Flink", "Kafka"], ["Flink", "YARN"], ["Flink", "Hive"], ["Flink", "HDFS"],
               ["Hive", "Spark"], ["Hive", "HBase"], ["Hive", "HDFS"], ["Hive", "Kafka"], ["Hive", "YARN"],
               ["HBase", "HDFS"], ["YARN", "HDFS"]]
upstream_sys_col = list()
downstream_sys_col = list()
freq_col = list()
for systems in system_order:
    upstream_sys_col.append(systems[0])
    downstream_sys_col.append(systems[1])
    freq_col.append(dic[systems[0] + "_" + systems[1]])
table1 = pd.DataFrame([upstream_sys_col, downstream_sys_col, freq_col]).transpose()
table1.columns = ['Upstream', 'Downstream','# CSI Failures']
table1

,Upstream,Downstream,# CSI Failures
0,Spark,Hive,26
1,Spark,YARN,19
2,Spark,HDFS,8
3,Spark,Kafka,5
4,Flink,Kafka,12
5,Flink,YARN,14
6,Flink,Hive,8
7,Flink,HDFS,3
8,Hive,Spark,6
9,Hive,HBase,3


## Section 5: Table 2

In [3]:
issues =  list(df['Failure Plane'])
total_issues = len(issues)
count = pd.Series(issues).value_counts()
planes = ["Control", "Data", "Management"]
vals = [count[plane] for plane in planes]
percentages = [str(round(count[plane]/total_issues * 100, 2)) + "%" for plane in planes]
table2 = pd.DataFrame([planes, vals, percentages]).transpose()
table2.columns = ['Plane', '# Failures','Percentage']
table2

,Plane,# Failures,Percentage
0,Control,20,16.67%
1,Data,61,50.83%
2,Management,39,32.5%


## Section 5:: Finding 3: Most (89/120) CSI failures are manifested through crashing behavior

In [4]:
planes =  list(df['Failure Plane'])
consequences = list(df['Failure Consequence'])
plane_to_consequences = defaultdict(lambda: defaultdict(int))
crashing_consequences = ["Runtime crash/hang", "Startup failure", "Job/task failure", "Job/task startup failure", "Job/task crash/hang"]
num_crashing_consequences = 0
for plane, consequence in list(zip(planes, consequences)):
    plane_to_consequences[plane][consequence.strip()] += 1
    if consequence in crashing_consequences:
        num_crashing_consequences += 1
print("Number of CSI failures that manifest through crashing behaviour: %s" % num_crashing_consequences)

## Section 5: Table 3

In [6]:
control_plane_rows = ["Runtime crash/hang", "Startup failure", "Performance issue", "Data loss", "Unexpected Behaviour"]
freq = list()
for consequence in control_plane_rows:
    freq.append(plane_to_consequences["Control"][consequence])
control_plane_table = pd.DataFrame([control_plane_rows, freq]).transpose()
control_plane_table.columns = ['Impact','#']
control_plane_table

,Impact,#
0,Runtime crash/hang,8
1,Startup failure,4
2,Performance issue,3
3,Data loss,2
4,Unexpected Behaviour,3


In [7]:
data_plane_rows = ["Job/task failure", "Job/task startup failure", "Wrong results", "Performance issues", "Resource leak", "Usability"]
freq = list()
for consequence in data_plane_rows:
    freq.append(plane_to_consequences["Data"][consequence])
data_plane_table = pd.DataFrame([data_plane_rows, freq]).transpose()
data_plane_table.columns = ['Impact','#']
data_plane_table

,Impact,#
0,Job/task failure,47
1,Job/task startup failure,6
2,Wrong results,3
3,Performance issues,2
4,Resource leak,2
5,Usability,1


In [8]:
management_plane_rows = ["Job/task crash/hang","Reduced observability", "Unexpected Behaviour", "Performance issue"]
freq = list()
for consequence in management_plane_rows:
    freq.append(plane_to_consequences["Management"][consequence])
management_plane_table = pd.DataFrame([management_plane_rows, freq]).transpose()
management_plane_table.columns = ['Impact','#']

management_plane_table

,Impact,#
0,Job/task crash/hang,24
1,Reduced observability,8
2,Unexpected Behaviour,5
3,Performance issue,2


In [9]:
planes =  list(df['Failure Plane'])
discrepancies = list(df['Discrepancy pattern type'])
discrepancy_to_count = defaultdict(lambda: defaultdict(int))
for plane, discrepancy in list(zip(planes, discrepancies)):
    discrepancy_to_count[plane][discrepancy] += 1
data_plane_discrepancy_rows = ["Type Confusion", "Unsupported Operations","Unspoken Convention", "Undefined Values", "Wrong API Assumptions"]
freq = list()
for discrepancy in data_plane_discrepancy_rows:
    freq.append(discrepancy_to_count["Data"][discrepancy])
table6 = pd.DataFrame([data_plane_discrepancy_rows, freq]).transpose()
table6.columns = ['Discrepancy Pattern','#']

control_plane_discrepancy_rows = ["API Semantics Violation", "State/Resource Inconsistency", "Feature inconsistency"]
freq = list()
for discrepancy in control_plane_discrepancy_rows:
    freq.append(discrepancy_to_count["Control"][discrepancy])
table7 = pd.DataFrame([control_plane_discrepancy_rows, freq]).transpose()
table7.columns = ['Discrepancy Pattern','#']


management_plane_discrepancy_rows = ["Ignorance", "Unexpected Override", "Inconsistent Context", "Mishandling Configuration Value"]
freq = list()
for discrepancy in management_plane_discrepancy_rows:
    freq.append(discrepancy_to_count["Management"][discrepancy])
table8 = pd.DataFrame([management_plane_discrepancy_rows, freq]).transpose()
table8.columns = ['Discrepancy Pattern','#']

## Section 6.1: Table 6

In [10]:
table6

,Discrepancy Pattern,#
0,Type Confusion,12
1,Unsupported Operations,15
2,Unspoken Convention,9
3,Undefined Values,7
4,Wrong API Assumptions,18


## Section 6.2: Table 7

In [11]:
table7

,Discrepancy Pattern,#
0,API Semantics Violation,13
1,State/Resource Inconsistency,5
2,Feature inconsistency,2


## Section 6.1: Table 8

In [12]:
table8

,Discrepancy Pattern,#
0,Ignorance,12
1,Unexpected Override,6
2,Inconsistent Context,10
3,Mishandling Configuration Value,2


## Section 7: Table 9

In [13]:
fix_patterns = list(df['Fix: Pattern'])
table9_rows = ["Checking", "Error handling", "Interaction", "Others"]
fix_pattern_to_count = defaultdict(int)
for pattern in fix_patterns:
    fix_pattern_to_count[pattern] += 1

freq = list()
for pattern in table9_rows:
    freq.append(fix_pattern_to_count[pattern])

table9 = pd.DataFrame([table9_rows, freq]).transpose()
table9.columns = ['Fix Pattern','#']
table9

,Fix Pattern,#
0,Checking,38
1,Error handling,8
2,Interaction,69
3,Others,5


## Finding 13: In 69% (79/115) CSI failures, fixes were applied to code in the upstream system specific to interaction with a downstream system. Furthermore, among these 79 cases, fixes for 68 (86%) cases resided in dedicated “connector” modules.

In [14]:
fix_locations = list(df['Fix: Location'])
fix_location_to_count = defaultdict(int)
for location in fix_locations:
    fix_location_to_count[location.strip()] += 1
code_specific_to_interaction_connector = fix_location_to_count["Code specific to downstream interaction: Connector"]
code_specific_to_interaction_scattered = fix_location_to_count["Code specific to downstream interaction: Scattered"]
code_specific_to_interaction = code_specific_to_interaction_connector + code_specific_to_interaction_scattered
no_fix_or_documentation = fix_location_to_count["N.A."]
total_fixes = len(fix_locations) - no_fix_or_documentation
print("Fixes applied to code in the upstream system specific to interaction " \
      "with a downstream system: %s (%s%%)" % (code_specific_to_interaction, round(code_specific_to_interaction/total_fixes * 100, 2)))
print("Fixes applied to code in the upstream system " \
      "in dedicated 'connector' modules: %s (%s%%)" % (code_specific_to_interaction_connector, round(code_specific_to_interaction_connector/total_fixes * 100, 2)))

Fixes applied to code in the upstream system specific to interaction with a downstream system: 79 (68.7%)
Fixes applied to code in the upstream system in dedicated 'connector' modules: 68 (59.13%)
